# Natural Language Processing | Recurrent Neural Networks


In [ ]:
import nltk
import numpy as np
import requests
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from nltk.corpus import treebank, brown, conll2000
from sklearn.model_selection import train_test_split
from tensorflow import keras

# Part-of-Speech Tagging with a Bidirectional LSTM

It's difficult to find free sequence labelling datasets because they're so labour-intensive to create.
<br><br>
Fortunately, **Natural Language Toolkit (NLTK)** includes enough free sets of labelled corpora for our purposes. NLTK also provides them in a convenient uniform format.<br>
https://www.nltk.org/index.html<br>
https://www.nltk.org/nltk_data/<br>
<br>
We'll use the Treebank, Brown, and CONLL-2000 datasets. 

In [ ]:
nltk.download('treebank')
nltk.download('brown')
nltk.download('conll2000')

In their original form, the datasets use different part-of-speech (PoS) tag sets. We need to ensure they all use the same tagset, so we'll download a simplified set called the *universal_tagset* from NLTK.<br>

See Section 2.3 here for a list of tags: https://www.nltk.org/book/ch05.html

In [ ]:
nltk.download('universal_tagset')

We'll then retrieve the tagged sentences from each dataset, taking care to specify they should use the *universal tagset* we just downloaded. We'll then combine them into one collection.

In [ ]:
# Download all PoS-tagged sentences and place them in one list.
tagged_sentences = treebank.tagged_sents(tagset='universal') +\
                   brown.tagged_sents(tagset='universal') +\
                   conll2000.tagged_sents(tagset='universal')

print(tagged_sentences[0])
print(f"Dataset size: {len(tagged_sentences)}")

Each tagged sentence is actually a list of word-tag tuples (bear in mind that NLTK's universal tagset is a reduced tagset so items such as *proper nouns* are simply tagged as *nouns*).<br>

Our model is going to take in a sequence of words, and output a sequence of PoS tags, so we need to separate the words from the tags in our dataset. The tag sequences will serve as our training labels.

In [ ]:
sentences, sentence_tags = [], []

for s in tagged_sentences:
  sentence, tags = zip(*s)
  sentences.append(list(sentence))
  sentence_tags.append(list(tags))

The sentences and their respective tags are now in separate lists.

In [ ]:
print(sentences[0])
print(sentence_tags[0])

In [ ]:
print(len(sentences), len(sentence_tags))

Create train/validation/test splits. This time, we don't have a separate test set so we'll call *train_test_split* twice.<br>
https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [ ]:
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

x_train, x_test, y_train, y_test = train_test_split(sentences, sentence_tags, 
                                                    test_size=1 - train_ratio, 
                                                    random_state=1)

x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, 
                                                test_size=test_ratio/(test_ratio + validation_ratio), 
                                                random_state=1)

In [ ]:
print(len(x_train), len(y_train))
print(len(x_val), len(y_val))
print(len(x_test), len(y_test))

Now that we have our datasets preprocessed, the next step is to vectorize. If you watched the demo on **Word Vectors**, the next few steps should look familiar.<br>
https://www.nlpdemystified.org/course/word-vectors<br>
https://colab.research.google.com/github/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_word_vectors.ipynb

First, we need to create a tokenizer for the sentences and *fit* it to the training dataset to create a vocabulary. We'll just use the default tokenizer settings which applies some light filtering, lowers the case, and separates on spaces. We'll also supply an out-of-vocabulary token (\<OOV\>) in case the tokenizer encounters words during testing/inference which it doesn't during training.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
sentence_tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<OOV>')

In [ ]:
sentence_tokenizer.fit_on_texts(x_train)

In [ ]:
print(f"Vocabulary size: {len(sentence_tokenizer.word_index)}")

We also need to create *another* tokenizer for the tags since our labels are also sequences. This time, we won't need an OOV token because there are only a handful of tags and, in this case, they'll all be encountered during training.

In [ ]:
tag_tokenizer = keras.preprocessing.text.Tokenizer()
tag_tokenizer.fit_on_texts(y_train)

In [ ]:
print(f"Number of PoS tags: {len(tag_tokenizer.word_index)}\n")
tag_tokenizer.get_config()

In [ ]:
# The set of universal PoS tags.
tag_tokenizer.word_index

Next, we need to vectorize our sentences and corresponding tags. As we did in the demo on [Word Vectors](https://github.com/futuremojo/nlp-demystified/blob/main/notebooks/nlpdemystified_word_vectors.ipynb), we'll use the tokenizer's *texts_to_sequences* method to convert each sentence to a sequence of integers where each integer maps to a particular token.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#texts_to_sequences

In [ ]:
x_train_seqs = sentence_tokenizer.texts_to_sequences(x_train)

In [ ]:
print(x_train_seqs[0])

We can use the *sequences_to_texts* method to convert a vectorized sentence back to its preprocessed form.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer#sequences_to_texts

In [ ]:
print(f"Original: {x_train[0]}")
print(f"Reconstructed: {sentence_tokenizer.sequences_to_texts([x_train_seqs[0]])}")

Next, we'll vectorize the labels (i.e. sequences of PoS tags) using its respective tokenizer.

In [ ]:
y_train_seqs = tag_tokenizer.texts_to_sequences(y_train)

In [ ]:
tag_tokenizer.sequences_to_texts([y_train_seqs[0]])

Finally, we'll do the same with the validation inputs and labels.

In [ ]:
x_val_seqs = sentence_tokenizer.texts_to_sequences(x_val)
y_val_seqs = tag_tokenizer.texts_to_sequences(y_val)

As we covered in the slides, **Recurrent Neural Networks** are capable of handling variable length sequences.<br><br>
Despite that, it's still best to pad or truncate sequences to a uniform length for one or both of these reasons:<br>
1. Performance. The longer a sequence, the higher the computation cost. One may want to truncate long sequences to a shorter length if that's feasible and doesn't result in too much performance loss.

2. When processing datasets in batches, each sequence *in a batch* usually has to be of uniform length.<br>

For simplicity, in this demo, we'll make *every* sequence be as long as the longest sequence. In other words, we'll determine how long the longest sequence is, then pad out the rest of the sequences to be the same length.<br>

A more optimized solution would be to make each sequence as long as the longest sequence in each *batch* to avoid unnecessary processing.

In [ ]:
MAX_LENGTH = len(max(x_train_seqs, key=len))
print(f"Length of longest input sequence: {MAX_LENGTH}")

We can pad the sequences with the *pad_sequences* method:<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

In [ ]:
x_train_padded = keras.preprocessing.sequence.pad_sequences(x_train_seqs, padding='post', 
                                                            maxlen=MAX_LENGTH)

In [ ]:
print(x_train_padded[0])

We'll do the same with the training label (PoS sequences)...

In [ ]:
y_train_padded = keras.preprocessing.sequence.pad_sequences(y_train_seqs, padding='post', 
                                                            maxlen=MAX_LENGTH)

...and the validation dataset.

In [ ]:
x_val_padded = keras.preprocessing.sequence.pad_sequences(x_val_seqs, padding='post', maxlen=MAX_LENGTH)
y_val_padded = keras.preprocessing.sequence.pad_sequences(y_val_seqs, padding='post', maxlen=MAX_LENGTH)

PoS tagging is a multiclass classification task done at each timestep, so we need to convert every tag for every sentence into a one-hot encoding (we'll look at an alternative approach when we build a language model later in this demo).<br>
https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical<br>

In [ ]:
y_train_categoricals = keras.utils.to_categorical(y_train_padded)

The label (PoS tag sequence) for a single sentence is now a **sequence of one-hot encodings**. These will serve as our training targets.

In [ ]:
# The one hot encodings for the first label.
print(y_train_categoricals[0])

In [ ]:
# One-hot encoding for a single tag.
print(y_train_categoricals[0][0])

We can determine the PoS tag from a one-hot encoding by seeing which index is set to 1, then using that to query the tag tokenizer's *index_word* dictionary.

In [ ]:
idx = np.argmax(y_train_categoricals[0][0])
print(f"Index: {idx}")

print(f"Tag: {tag_tokenizer.index_word[idx]}")

We'll one-hot encode the validation labels as well.

In [ ]:
y_val_categoricals = keras.utils.to_categorical(y_val_padded)

At this point, we're ready to build our model. We'll train word embeddings concurrently with our model (though you can use pretrained word vectors as well). If you're unfamiliar with this idea, refer to the [Word Vectors](https://www.nlpdemystified.org/course/word-vectors) module.<br><br>
There are several new things here:<br>
1. The embedding layer has a *mask_zero* parameter. We added padding in order to make our batches the same size, but we don't want the model to make PoS predictions on padding. Setting *mask_zero* to *True* makes the layers following the embedding layer ignore padding values.<br>
https://www.tensorflow.org/guide/keras/masking_and_padding<br>
https://stackoverflow.com/questions/47485216/how-does-mask-zero-in-keras-embedding-layer-work<br><br>
2. We're using a **bidirectional LSTM**. The *Bidirectional* layer is a wrapper to which we pass an *LSTM* layer. The first parameter to the *LSTM* layer is the number of units in the cell. The second parameter, *return_sequences*, controls whether the RNN returns an output for each timestep or only the last output. Since we're doing PoS-tagging, we want an output for each timestep and so *return_sequences* is set to *True*.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM<br>
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional


In [ ]:
# For the embedding layer. "+ 1" to account for the padding token.
num_tokens = len(sentence_tokenizer.word_index) + 1
embedding_dim = 128

# For the output layer. The number of classes corresponds to the 
# number of possible tags.
num_classes = len(tag_tokenizer.word_index) + 1

In [ ]:
# The set_seed call and kernel_initializer parameters are used here to
# ensure you and I get the same results. To get random weight initializations,
# remove them.
tf.random.set_seed(0)

model = keras.Sequential()

model.add(layers.Embedding(input_dim=num_tokens, 
                           output_dim=embedding_dim, 
                           input_length=MAX_LENGTH,
                           mask_zero=True))

model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True, 
                                           kernel_initializer=tf.keras.initializers.random_normal(seed=1))))

model.add(layers.Dense(num_classes, activation='softmax', 
                       kernel_initializer=tf.keras.initializers.random_normal(seed=1)))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


A few notes about the model summary:<br>

The embedding layer **output** has three dimensions:
- Batch size (it's showing as "None" because we didn't specify it upfront. We'll do it when we call *model.fit*).
- Sequence length (the sequences are all the same length now after our padding step).
- Embedding dimension.
<br><br>

The LSTM outputs a vector *twice* the size of what we specified because it's bidirectional. Recall from the slides that the outputs from the two LSTMs will be concatenated before going to the output layer.
<br><br>

The final layer's **output** also has three dimensions:
- Batch size
- Sequence length
- Output dimension (the number of possible tags).

The output will be a **sequence of probability distributions** for each input sequence. One probability distribution per tag.



In [ ]:
model.summary()

As we did in previous demos, we'll use *early stopping* with some *patience* to halt training once validation loss stops improving.<br>

The model will compare its output (sequences of softmax-generated probability distributions) against the one-hot encoded targets.



In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

history = model.fit(x_train_padded, y_train_categoricals, epochs=20, 
                    batch_size=256, validation_data=(x_val_padded, y_val_categoricals), 
                    callbacks=[es_callback])

Once our model is trained, we'll vectorize and pad the testing dataset. In the case of the labels, we'll also one-hot encode them.

In [ ]:
# Preprocess the test data and test the model.
x_test_seqs = sentence_tokenizer.texts_to_sequences(x_test)
x_test_padded = keras.preprocessing.sequence.pad_sequences(x_test_seqs, padding='post', maxlen=MAX_LENGTH)

y_test_seqs = tag_tokenizer.texts_to_sequences(y_test)
y_test_padded = keras.preprocessing.sequence.pad_sequences(y_test_seqs, padding='post', maxlen=MAX_LENGTH)
y_test_categoricals = keras.utils.to_categorical(y_test_padded)

In [ ]:
model.evaluate(x_test_padded, y_test_categoricals)

We can now use our model to tag sentences.

In [ ]:
samples = [
    "Brown refused to testify.",
    "Brown sofas are on sale.",
]

The function below takes a list of strings, tokenizes and pads them, then has the model tag them. Note that if a sentence is longer than MAX_LENGTH, it'll be truncated.

In [ ]:
def tag_sentences(sentences):
  sentences_seqs = sentence_tokenizer.texts_to_sequences(sentences)
  sentences_padded = keras.preprocessing.sequence.pad_sequences(sentences_seqs, 
                                                                maxlen=MAX_LENGTH, 
                                                                padding='post')

  # The model returns a LIST of PROBABILITY DISTRIBUTIONS (due to the softmax)
  # for EACH sentence. There is one probability distribution for each PoS tag.
  tag_preds = model.predict(sentences_padded)

  sentence_tags = []

  # For EACH LIST of probability distributions...
  for i, preds in enumerate(tag_preds):

    # Extract the most probable tag from EACH probability distribution.
    # Note how we're extracting tags for only the non-padding tokens.
    tags_seq = [np.argmax(p) for p in preds[:len(sentences_seqs[i])]]

    # Convert the sentence and tag sequences back to their token counterparts.
    words = [sentence_tokenizer.index_word[w] for w in sentences_seqs[i]]
    tags = [tag_tokenizer.index_word[t] for t in tags_seq]
    sentence_tags.append(list(zip(words, tags)))

  return sentence_tags


In [ ]:
tagged_sample_sentences = tag_sentences(samples)

In [ ]:
print(tagged_sample_sentences[0])

In [ ]:
print(tagged_sample_sentences[1])

So that's one way to build a PoS tagger. Industrial-strength taggers use a lot more data and these days, are powered by more sophisticated models which we'll learn about when we cover *transformers*.

# Language Modelling With Stacked LSTMs

We'll build a language model trained on the *Art of War* by Sun Tzu.

In [ ]:
art_of_war = requests.get('https://raw.githubusercontent.com/futuremojo/nlp-demystified/main/datasets/art_of_war.txt')\
                     .text

art_of_war[:300]

The language model we'll build will be **character**-based (as opposed to word-based). That is, given a sequence of one or more characters, the model will be asked to predict the next character.<br><br>
Character-level models have the advantage of:
- Smaller prediction space. There are only a handful of characters in the English language compared to the tens of thousands of words in a typical corpus.
- Character-level models are more resilient to out-of-vocabulary (OOV) conditions and are better able to learn the lower mechanics of language (including punctuation).<br><br>

On the other hand, character-level models need to learn a sequence of characters to "make sense" of a word (e.g. the sequence of "c", "a", "t" to identify "cat" as a pattern) which can be inefficient and result in lower performance.<br><br>
RNNs can process any kind of sequence so what's shown here can easily be applied at the word level. When we cover *transformers*, we'll learn an alternative tokenization technique called **subword tokenization** that is a middle-ground between these two approaches.

We'll initialize a Keras **Tokenizer** and set the *char_level* parameter to True so that our corpus gets tokenized into characters rather than words. It'll continue lower-casing and applying its default light filtering.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)

In [ ]:
tokenizer.fit_on_texts([art_of_war])

The tokenizer's internal dictionary now maps characters rather than words...

In [ ]:
tokenizer.get_config()

...and the resulting possibility space is much smaller.

In [ ]:
print(f"Tokenizer \"Vocabulary\" size: {len(tokenizer.word_index)}")

As we did when building the PoS tagger, we'll vectorize the book's characters into a sequence of integers, each integer mapping to a particular character.

In [ ]:
seq = tokenizer.texts_to_sequences([art_of_war])[0]

In [ ]:
print(f"Text length: {len(seq)}")

In [ ]:
# Sanity check.
tokenizer.sequences_to_texts([seq[:10]])

Our training data is currently one long sequence which we'll need to segment into training examples. To do this, we'll use the **Tensorflow Data** API which makes it easy to build preprocessing pipelines by chaining operations together.<br>
https://www.tensorflow.org/guide/data<br>
https://www.tensorflow.org/api_docs/python/tf/data<br>

To use this API, we need to first convert our vectorized corpus (which is a plain Python list) into a  **Dataset** object, which we can do using the *from_tensor_slices* method. This takes our vectorized corpus and returns a stream of tensors, one tensor for each integer. We'll then be able to perform operations on this **Dataset** object to prep our data.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset

In [ ]:
slices = tf.data.Dataset.from_tensor_slices(seq)
type(slices)

Like Python **generators**, TF **Datasets** function like iterators, so we need to do things like convert them into a list or iterate over them in order to print them.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take

In [ ]:
# Create a Dataset from the first ten slices and convert to a list to output to console.
list(slices.take(10))

In [ ]:
# The first ten elements from the original vectorized corpus.
seq[:10]

The next step is to create the training examples. To do that, we'll use the *window* method. Calling *window* on a dataset results in a sequence of datasets each containing N elements. So calling window(100) on a dataset of 1000 will result in a sequence of 10 datasets, each containing 100 elements.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#window

Here, we're creating windows of `input_timesteps + 1`. The *input_timesteps* represents our training example length. The *+1* is there to help us create the target/label for each training example. This will be clarified further below.<br><br>
In addition, we're setting *shift* to 1. This means we'll get overlapping windows shifted by 1. e.g. if the input is [1, 2, 3, 4, ...]. The first window will contain [1, 2, 3, ...], the second window will contain [2, 3, 4, ...] and so on. This is so we can have more training examples.<br><br>
Finally, we're setting *drop_remainder* to True which ensures ALL windows contain exactly N elements. i.e. once the input contains fewer than N elements, they are ignored.

In [ ]:
input_timesteps = 100
window_size = input_timesteps + 1
windows = slices.window(window_size, shift=1, drop_remainder=True)

Looking at the first few windows, we can see they're all the same length and that each subsequent window is shifted over by 1. Our corpus has now been divided into segments of length `input_timesteps + 1`.

In [ ]:
for w in windows.take(3):
  arr = list(w.as_numpy_iterator())
  print(len(arr), arr)

The *window* method returns a nested dataset of datasets (i.e. each window is a dataset containing a tensor).

In [ ]:
print(windows, '\n')

for w in windows.take(2):
  print(w)

But our model won't accept those. It'll accept only tensors, so we need to extract the tensors from each window. To do that, we'll use *flat_map* which will flatten the dataset of datasets into a single dataset of elements. But because we want to retain our segmented sequences, we'll also pass in a *batch* function to maintain the segments (otherwise, we'll just get back one large tensor representing our whole corpus).<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#flat_map<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#batch

In [ ]:
dataset = windows.flat_map(lambda window: window.batch(window_size))

We now have a single dataset of tensors, where each tensor is `input_timesteps+1` long and shifted by 1.

In [ ]:
for d in dataset.take(2):
  print(d)

The next step is to create batches from our dataset. To do this, we'll shuffle the dataset, then create batches.<br>
https://www.tensorflow.org/api_docs/python/tf/data/Dataset#shuffle<br>


In [ ]:
batch_size = 32

In [ ]:
# The expected number of batches should be (len(seq) - input_timesteps) / batch_size
batches = dataset.shuffle(10000).batch(batch_size)

In [ ]:
for b in batches.take(2):
  print(b)

We can now separate each example into an input sequence(x) and a corresponding label/target sequence(y).<br><br>
In the slides, we talked about **Teacher Forcing** where:<br>
1. At each timestep during training, the output is compared to a label.
2. At the next timestep, rather than feeding the model the previous output, we feed it the next character of the input sequence (i.e. what the model should've outputted).
<br><br>

This is why each sequence is of size *input_timesteps + 1*. Each sequence is now going to be separated into TWO sequences. The first sequence will be the training input and will be of length *input_timesteps* (i.e. everything but the LAST character). The second sequence will be the label/target and will consist of all the sequence elements shifted by 1 (i.e. everything but the FIRST character). If this is confusing, refer to the accompanying video for this module where we cover training language models.<br><br>

So if a sequence is "she swam in the lake", then:
- The input will be "she swam in the lak" (drop the last character)
- The target/label will be "he swam in the lake" (drop the first character)

In [ ]:
xy_batches = batches.map(lambda batch: (batch[:, :-1], batch[:, 1:]))

Each batch now consists of a set of input sequences and a corresponding set of label/target sequences, with the labels/targets shifted by 1.

In [ ]:
for b in xy_batches.take(1):
  print(b)

In [ ]:
# For greater clarity, this is the first input sequence from the first batch,
# and it's corresponding label/target sequence.
for b in xy_batches.take(1):
  print("x1 length: ", len(b[0][0].numpy()))
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1 length: ", len(b[1][0].numpy()))
  print("y1: ", b[1][0].numpy())

The last step before we can build our model is to one-hot encode the **inputs**. We're doing this because:
1. We're not using embeddings for the input. We can, but since this is a character model with just a few dozen possible choices, we can get away with one-hot encoding. There's also no reason to think a particular letter should be closer to another in vector space as we would want in a word-level model.

2. Since we're not using embeddings and our input is categorical, we need to one-hot encode.

Note that despite our labels ALSO being categorical, we are NOT one-hot encoding them this time. This is because we'll be using a loss function that can help us skip that step (more below).

In [ ]:
num_tokens = len(tokenizer.word_index) + 1

# One-hot encode the input sequences, don't do anything with the label/target sequences.
xy_batches = xy_batches.map(lambda inputs, labels: (tf.one_hot(inputs, depth=num_tokens), labels))

In [ ]:
# Each input sequence is now a sequence of one-hot encodings.
for b in xy_batches.take(1):
  print("x1: ", b[0][0].numpy())
  print("\n")
  print("y1: ", b[1][0].numpy())

At this point, we've:
- Segmented our corpus into fixed-length sequences.
- Created training and label/target sequences.
- Organized them into batches.

The last step is to add some **prefetching**. This is an optimization step. This way, while the model trains on the current batch of data, the pipeline reads and prepares the next batch.<br>
https://www.tensorflow.org/guide/data_performance#prefetching

In [ ]:
dataset = dataset.prefetch(tf.data.AUTOTUNE)

We can now build our model. There are three new things here:
1. We're stacking two LSTMs. The sequential output of the first LSTM will become the sequential input to the second LSTM.<br><br>
2. We're adding some *recurrent_dropout*. This drops connections between the recurrent units (i.e. the dropout is applied horizontally across time). You can still use regular *dropout* as well which will be applied to the inputs/outputs. Refer to this paper for more information:<br>
https://arxiv.org/abs/1512.05287<br><br>
3. We're using **sparse_categorical_crossentropy**. This allows us to provide labels as integers for multiclass classification rather than one-hot encodings.<br>
https://keras.io/api/losses/probabilistic_losses/#sparsecategoricalcrossentropy-class

In [ ]:
model = keras.models.Sequential()

model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=True, input_shape=[None, num_tokens], recurrent_dropout=0.2))

model.add(layers.Dense(num_tokens, activation='softmax'))

model.compile(loss="sparse_categorical_crossentropy", optimizer='adam')


In [ ]:
model.summary()

Because this model takes a few hours to train, we're using **model checkpoints** to save the weights after every epoch. This way, if something goes wrong with our system during training, we can reload the last set of weights from the checkpoint, and resume training from there.<br>
https://keras.io/api/callbacks/model_checkpoint/

In [ ]:
filepath="./ArtofWarLM/training1/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=filepath,
                                                 save_weights_only=True,
                                                 verbose=1)

When calling the model's *fit* method, we:<br>
1. simply pass in the batches as is (no need to separate into explicit x and y arguments).
2. pass the model checkpoint callback to save the weights after every epoch

**NOTE**: The call to *fit* below is commented out. Because this model takes a few hours to train, I trained it ahead of time and saved it. If you want to train it yourself, feel free to uncomment and execute it. Also, because of the random weight initialization, your trained model's output will likely differ from mine.

In [ ]:
#
# history = model.fit(xy_batches, epochs=50, callbacks=[cp_callback])
#

Once training is complete, we can call the model's *save* method to save its weights and metadata. Again, this is commented out because I already trained the model previously.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/Model#save<br>
https://www.tensorflow.org/guide/keras/save_and_serialize

In [ ]:
#
# model.save('art_of_war_char_level_lm')
#

Download and unzip the previously trained model...

In [ ]:
!wget https://github.com/futuremojo/nlp-demystified/raw/main/models/art_of_war_char_level_lm.zip
!unzip -o art_of_war_char_level_lm.zip

...and load it.<br>
https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model

In [ ]:
model = keras.models.load_model('art_of_war_char_level_lm')

Now that we have a trained model, let's generate some text.<br><br>
The function below takes some seed text and uses that to generate a certain number of characters. For each character, it uses the generated text so far as the input. It's not the most efficient function but it'll work here.<br><br>
There's also a *temperature* parameter. The next character is picked from a probability distribution. By dividing the log of this distribution by *temperature*, we can influence the randomness of the output.<br><br>
When the temperature is low (< 1), the probability distribution sharpens and the model will be more strict in recreating the original text. As we raise the temperature, the distribution flattens and there's a higher chance the model picks something unexpected, resulting in greater surprise in the output. In practice, a high enough temperature will result in nonsense.

In [ ]:
def generate_text(model, tokenizer, seed_text, num_chars=200, temperature=1):

  text = seed_text  

  for _ in range(num_chars):

    # Take the last *input_timesteps* number of characters in the text so far
    # as input.
    input = np.array(tokenizer.texts_to_sequences([text[-input_timesteps:]]))
    input = tf.one_hot(input, num_tokens)

    # Create probability distribution for next character adjusted by temperature.
    preds = model.predict(input)[0, -1:, :] # <-- We want only the last character so we're extracting the softmax output for that.
    preds = tf.math.log(preds) / temperature

    # Sample next character and add to running text.
    next_char = tf.random.categorical(preds, num_samples=1)
    next_char = tokenizer.sequences_to_texts(next_char.numpy())[0]

    text += next_char

  return text


In [ ]:
%%time
print(generate_text(model, tokenizer, "Banana peels on the battlefield can", num_chars=300, temperature=0.2))

In [ ]:
print(generate_text(model, tokenizer, "It's time to release the Kraken when", num_chars=300, temperature=0.5))

In [ ]:
print(generate_text(model, tokenizer, "Crush your enemies, see them driven before you, and", num_chars=300, 
                    temperature=1))

In [ ]:
print(generate_text(model, tokenizer, "What is best in life?", num_chars=300, temperature=2))

A few observations of the preceding outputs:
1. Despite being a character-level model, the model managed to "learn" spelling, cadence, punctuation, spacing, grammar, and even numbered bullet points just from trying to predict the next character.

2. It's pretty cool how the model manages to take our initial seed text and complete a sentence with it before moving on.

3. We can see the output getting increasingly nonsensical as the temperature rises. What temperature to use ultimately depends on the nature of your corpus and your goals with the language model.

Also, in contrast to our language model, GPT-3 has 175 billion parameters and was trained on 45 terabytes of data, but the high-level principle of learning through prediction remains the same.

# Further Exploration
1. Check out *Sunspring*, a sci-fi short written by an LSTM. The director and actors played the script straight and the result is hilarious.<br>
https://www.youtube.com/watch?v=LY7x2Ihqjmc<br>
https://en.wikipedia.org/wiki/Sunspring<br><br>
2. Everything we learned here can be applied at the word-level. Try creating a word-level language model with a different corpus (maybe download something from https://www.gutenberg.org/) and try using word embeddings.<br><br>
3. We didn't evaluate our language model using perplexity. Find out online how to do it.